# Pre-processing Texas TCEQ Allocation data for WaDEQA upload.
Date Updated: 03/28/2023
Purpose:  To pre-process the Texas data into one master file for simple DataFrame creation and extraction

In [ ]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [ ]:
#Working Directory
workingDir = "G:/Shared drives/WaDE Data/Texas/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Assign Owner Name
- create dictionary of owner names
- will use a dictionary to assign owner name for output main file.

In [ ]:
#Dataframe creation - owners
ownerInput = "WaterRightOwner.csv"
df_owner = pd.read_csv(ownerInput, usecols=['Water Right ID', 'Owner'], encoding="ISO-8859-1")
df_owner.head()

In [ ]:
# Clean Owner info.  Remove special characters

import re

def cleanOwnerDataFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().strip()
    return Val

df_owner['Owner'] = df_owner.apply(lambda row: cleanOwnerDataFunc(row['Owner']), axis=1)
df_owner['Owner'].unique()

In [ ]:
# Merge onwers, assign ID
# Issue of multiple owners per water Right ID

def retrieveNames(df):
    ids = df['Water Right ID'].drop_duplicates()
    outdf = pd.DataFrame(ids)
    outdf.reset_index(drop=True, inplace=True)
    outdf['owners'] = ''
    outdf.set_index(outdf['Water Right ID'], inplace=True)

    for id in ids:
        vals = df.loc[df['Water Right ID'] == id]
        vals.reset_index(inplace=True)
        names = []
        for i, row in vals.iterrows():
            names.append(row['Owner'])

        outdf.at[id, 'owners'] = ', '.join(names)

    return outdf

df_owners = retrieveNames(df_owner)
df_owners['WaterRightID'] = df_owners['Water Right ID']
df_owners.head()

## Assign Ben Use
- create dictionary of ben use
- will use a dictionary to assign ben use for output main file.

In [ ]:
#Dataframe creation - ben use
useInput = "WaterUse.csv"
df_use = pd.read_csv(useInput, usecols=['Water Right ID', 'Use'])
df_use['Use'] = df_use['Use'].str.title()
df_use.head()

In [ ]:
def retrieveUses(df):
    ids = df['Water Right ID'].drop_duplicates()
    outdf = pd.DataFrame(ids)
    outdf.reset_index(drop=True, inplace=True)
    outdf['uses'] = ''
    outdf.set_index(outdf['Water Right ID'], inplace=True)

    for id in ids:
        vals = df.loc[df['Water Right ID'] == id]
        vals.reset_index(inplace=True)
        uses = []
        for i, row in vals.iterrows():
            new_use = row['Use']

            if new_use not in uses:
                uses.append(new_use)

        outdf.at[id, 'uses'] = ','.join(uses)

    return outdf

df_uses = retrieveUses(df_use)
df_uses['WaterRightID'] = df_uses['Water Right ID']
df_uses.head()

## POD: Water Right Points and Output file
- maint output

In [ ]:
# Input File
fileInput = "WaterRightPoint.csv"
dfinPOD = pd.read_csv(fileInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOD:
    dfinPOD['WaDEUUID'] = "txWR" + dfinPOD.index.astype(str)
    dfinPOD.to_csv('WaterRightPoint.zip', compression=dict(method='zip', archive_name='WaterRightPoint.csv'), index=False)

print(len(dfinPOD))
dfinPOD.head()

In [ ]:
# Assining owner name to output file

# Loop up dictonary using owner dataframe
OwnerDict = pd.Series(df_owners.owners.values, index=df_owners.WaterRightID).to_dict()

def retrieveOwner(val):
    if val == '' or pd.isnull(val):
        outString = "WaDE Unspecified"
    else:
        String1 = str(val).strip()
        try:
            outString = OwnerDict[String1]
        except:
            outString = "WaDE Unspecified"
    return outString

dfinPOD['in_AllocationOwner'] = dfinPOD.apply(lambda row: retrieveOwner(row['WR_ID']), axis=1)
dfinPOD['in_AllocationOwner'].unique()

In [ ]:
# Assining ben use to output file

# Loop up dictonary using ben use dataframe
BenuseDict = pd.Series(df_uses.uses.values, index=df_uses.WaterRightID).to_dict()

def retrieveBenUse(val):
    if val == '' or pd.isnull(val):
        outString = "WaDE Unspecified"
    else:
        String1 = str(val).strip()
        try:
            outString = BenuseDict[String1]
        except:
            outString = "WaDE Unspecified"
    return outString

dfinPOD['in_BeneficialUseCategory'] = dfinPOD.apply(lambda row: retrieveBenUse(row['WR_ID']), axis=1)
dfinPOD['in_BeneficialUseCategory'].unique()

In [ ]:
# Tx projection = EPSG:4269.  WGS84 projection used by WaDE 2.0 = epsg:4326.

from pyproj import Transformer, transform # for transforming coordinates to a new projection
transformer = Transformer.from_proj(4269, 4326)  

def assignLat(colrowValueLat, colrowValueLong):
    lat, long = transformer.transform(colrowValueLat, colrowValueLong)
    return lat

def assignLong(colrowValueLat, colrowValueLong):
    lat, long = transformer.transform(colrowValueLat, colrowValueLong)
    return long



dfinPOD['in_Latitude'] = dfinPOD.apply(lambda row: assignLat(row['LAT_DD'], row['LONG_DD']), axis=1)
dfinPOD['in_Longitude'] = dfinPOD.apply(lambda row: assignLong(row['LAT_DD'], row['LONG_DD']), axis=1)
dfinPOD.head()

In [ ]:
#fixing Spelling issues in TCEQ TYPE field
TYPEdict = {
"Dischrage Point" : "Discharge Point",
"Dishcharge Point" : "Discharge Point",
"IBT -  Diversion Point" : "IBT - Diversion Point",
"On-channel  Reservoir" : "On-channel Reservoir",
"On-channel Reservior" : "On-channel Reservoir",
"GW -  Release Point" : "GW - Release Point"
}

def updateTYPE(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = colrowValue
    else:
        String1 = colrowValue  # remove whitespace chars
        try:
            outList = TYPEdict[String1]
        except:
            outList = colrowValue
    return outList

dfinPOD['TYPE'] = dfinPOD.apply(lambda row: updateTYPE(row['TYPE']), axis=1)
dfinPOD['TYPE'].unique()

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOD['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "TXwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "TXwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "TXwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = "WaDE Unspecified"
df['in_WaterSourceNativeID'] = "" # autfo fill in below
df['in_WaterSourceTypeCV'] = "WaDE Unspecified"

# Site Info
df['in_CoordinateAccuracy'] = "WaDE Unspecified"
df['in_CoordinateMethodCV'] = "Digitized"
df['in_County'] = "WaDE Unspecified"
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOD['in_Latitude']
df['in_Longitude'] = dfinPOD['in_Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = "WaDE Unspecified"
df['in_SiteNativeID'] = "POD" + dfinPOD['TCEQ_ID'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfinPOD['TYPE']
df['in_StateCV'] = "TX"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = "" #empty
df['in_AllocationLegalStatusCV'] = "WaDE Unspecified"
df['in_AllocationNativeID'] =  dfinPOD['WR_ID'].replace("", 0).fillna(0).str.strip().astype(str)
df['in_AllocationOwner'] = dfinPOD['in_AllocationOwner']
df['in_AllocationPriorityDate'] = "" #empty
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = "" #empty
df['in_BeneficialUseCategory'] = dfinPOD['in_BeneficialUseCategory']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 1 # we want this data excempt
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "https://gisweb.tceq.texas.gov/WRRetrieveRights/?ID=" + dfinPOD['WR_TYPE_NO'].replace("", 0).fillna(0).str.strip().astype(str)

outPOD = df.copy()
outPOD = outPOD.drop_duplicates().reset_index(drop=True)
print(len(outPOD))
outPOD.head()

In [ ]:
# Concatenate dataframes
frames = [outPOD]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

## Inspect and Clean Data

In [ ]:
# Fixing empty string names

def fixEmptyString(val):
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = "WaDE Unspecified"
    else:
        outString = val
    return outString

In [ ]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: fixEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

In [ ]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: fixEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

In [ ]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: fixEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

In [ ]:
# in_Latitude & in_Longitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna(0)
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna(0)
outdf.head(1)

In [ ]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'])
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

In [ ]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').fillna(0)
outdf['in_AllocationFlow_CFS'].unique()

In [ ]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').fillna(0)
outdf['in_AllocationVolume_AF'].unique()

In [ ]:
%%time

# Creating WaDE Custom water source native ID for easy water source identification
# Change 'outstring' name to be state specific.
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A) & 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

## Export Data

In [ ]:
# Export the output dataframe
outdf.to_csv('Pwr_txMain.zip', index=False, compression="zip")  # The output, save as a zip
#dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.